In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import missingno

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [10]:
data = pd.read_csv('data\premier league\england-premier-league-matches-2023-to-2024-stats.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 67 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   timestamp                            380 non-null    int64  
 1   date_GMT                             380 non-null    object 
 2   status                               380 non-null    object 
 3   attendance                           66 non-null     float64
 4   home_team_name                       380 non-null    object 
 5   away_team_name                       380 non-null    object 
 6   referee                              379 non-null    object 
 7   Game Week                            380 non-null    int64  
 8   Pre-Match PPG (Home)                 380 non-null    float64
 9   Pre-Match PPG (Away)                 380 non-null    float64
 10  home_ppg                             380 non-null    float64
 11  away_ppg                        

In [5]:
data.head()

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1691780400,Aug 11 2023 - 7:00pm,complete,21572.0,Burnley,Manchester City,Craig Pawson,1,0.0,0.0,0.53,2.32,0,3,3,2,0,2,NaN,"4,36,75",6,5,0,1,0,0,0,1,0,0,5,13,2,9,3,4,11,8,34,66,0.0,0.0,0.65,1.72,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,11.00,6.30,1.28,1.20,1.56,2.65,4.85,2.05,1.70,Turf Moor (Burnley),premier league
1,1691841600,Aug 12 2023 - 12:00pm,complete,59984.0,Arsenal,Nottingham Forest,Michael Oliver,1,0.0,0.0,2.47,0.84,2,1,3,2,2,0,"26,32",82,8,3,2,0,2,0,1,1,1,1,11,6,8,3,3,3,12,12,78,22,0.0,0.0,1.72,0.76,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.25,6.25,9.50,1.11,1.45,2.01,3.45,2.00,1.75,Emirates Stadium (London),premier league
2,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,AFC Bournemouth,West Ham United,Peter Bankes,1,0.0,0.0,1.42,1.21,1,1,2,0,0,0,82,51,10,4,1,0,4,0,0,1,1,3,13,13,6,4,7,9,8,14,63,37,0.0,0.0,1.66,1.39,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.63,3.06,2.35,1.29,1.89,3.00,5.75,1.70,2.05,Vitality Stadium (Bournemouth- Dorset),premier league
3,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,Brighton & Hove Albion,Luton Town,David Coote,1,0.0,0.0,1.58,0.53,4,1,5,1,1,0,"36,71,85,90'5",81,6,7,2,0,2,0,2,0,0,2,23,8,13,4,10,4,4,8,71,29,0.0,0.0,2.91,1.06,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.34,4.70,7.00,1.18,1.58,2.38,4.90,2.00,1.75,The American Express Community Stadium (Falmer...,premier league
4,1691848800,Aug 12 2023 - 2:00pm,complete,NaN,Everton,Fulham,Stuart Attwell,1,0.0,0.0,1.47,0.95,0,1,1,0,0,0,NaN,73,10,4,0,0,2,0,0,0,1,1,16,9,10,3,6,6,12,6,41,59,0.0,0.0,2.09,1.05,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.32,3.10,2.89,1.29,2.08,3.40,7.00,1.80,1.95,Goodison Park (Liverpool),premier league


In [11]:
data = clean_data(data)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_2548\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [33]:
data_folder = './data'

for entry in os.listdir(data_folder):
    entry_path = os.path.join(data_folder, entry)
    if os.path.isdir(entry_path):
        for file_name in os.listdir(entry_path):
            if file_name.endswith('.csv'):
                file_path = os.path.join(entry_path, file_name)
                df = pd.read_csv(file_path)
                df['league'] = entry
                df.to_csv(file_path, index=False)

In [34]:
data = pd.DataFrame()

for folder in os.listdir('./data'):
    if folder.endswith('csv'):
        continue
    for file in os.listdir(f'./data/{folder}'):
        df = pd.read_csv(f'./data/{folder}/{file}')
        data = pd.concat([data, df], ignore_index=True)

In [35]:
data.isna().sum()

timestamp                                  0
date_GMT                                   0
status                                     0
attendance                             30917
home_team_name                             0
away_team_name                             0
referee                                31947
Game Week                               6853
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_goal_timings                 19417
away_team_goal_timings                 27086
home_team_corner_count                     0
away_team_corner_count                     0
home_team_

In [7]:
def clean_data(data):
    #transforming date variable
    data['date_GMT'] = pd.to_datetime(data['date_GMT'])
    data['date'] = pd.to_datetime(data['date_GMT'].dt.date)
    data['time'] = data['date_GMT'].dt.time
    
    #adding aggregated variables
    data['corners_total'] = data['home_team_corner_count'] + data['away_team_corner_count']
    data['yellow_cards_total'] = data['home_team_yellow_cards'] + data['away_team_yellow_cards']
    data['red_cards_total'] = data['home_team_red_cards'] + data['away_team_red_cards']
    data['cards_total'] = data['yellow_cards_total'] + data['red_cards_total']
    data['shots_total'] = data['home_team_shots'] + data['away_team_shots']
    data['shots_on_target_total'] = data['home_team_shots_on_target'] + data['away_team_shots_on_target']
    data['shots_off_target_total'] = data['home_team_shots_off_target'] + data['away_team_shots_off_target']
    data['fouls_total'] = data['home_team_fouls'] + data['away_team_fouls']
    
    #getting stadium names without city in brackets
    # data['base_name'] = data['stadium_name'].str.replace(r" \(.*\)$", "", regex=True)

    # city_map = data[data['stadium_name'].str.contains(r"\(.*\)")].copy()
    # city_map['city'] = city_map['stadium_name'].str.extract(r"\((.*?)\)")[0]
    # city_map = city_map.groupby('base_name')['city'].agg(pd.Series.mode).to_dict()

    # #filling stadium names without city in brackets
    # data['normalized_stadium'] = data.apply(lambda row: f"{row['base_name']} ({city_map.get(row['base_name'], 'Unknown')})" if '(' not in row['stadium_name'] else row['stadium_name'], axis=1)

    #dropping unnecessary features
    data.drop(['timestamp', 'status', 'home_team_goal_timings', 'away_team_goal_timings', 'date_GMT'], axis=1, inplace=True)
    
    #setting one of the target variable - result of the game
    data['result'] = np.where(data['home_team_goal_count'] == data['away_team_goal_count'], 0, np.where(data['home_team_goal_count'] > data['away_team_goal_count'], 1, 2))
    
    def assign_season(date):
        year = date.year
        if date.month >= 8:  # Sezon zaczyna się w sierpniu
            return f'{str(year)[2:]}/{str(year+1)[2:]}'
        else:
            return f'{str(year-1)[2:]}/{str(year)[2:]}'
    
    data['season'] = data['date'].apply(assign_season)
    
    return data

In [39]:
data = clean_data(data)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_23700\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [40]:
data.shape

(83142, 74)

In [41]:
data.head()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season
0,8016.0,Austria Wien,Admira,NaN,1.0,0.0,0.0,1.61,0.67,2,0,2,0,0,0,6,7,2,0,5,0,0,2,1,4,13,13,7,6,6,7,9,17,53,47,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.36,5.15,9.65,0.0,0.0,0.0,0.0,0.0,0.0,Generali Arena (Wien),austria,2013-07-20,14:30:00,13,7,0,7,26,13,13,26,1,12/13
1,2840.0,Grödig,Ried,NaN,1.0,0.0,0.0,1.44,0.94,0,0,0,0,0,0,6,11,8,0,5,0,5,3,2,3,8,16,4,10,4,6,20,24,49,51,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.70,3.42,2.73,0.0,0.0,0.0,0.0,0.0,0.0,DAS.GOLDBERG Stadion,austria,2013-07-20,17:00:00,17,13,0,13,24,14,10,44,0,12/13
2,2696.0,Wiener Neustadt,Salzburg,NaN,1.0,0.0,0.0,1.11,1.94,1,5,6,3,1,2,3,10,4,0,1,0,0,4,0,1,5,18,2,6,3,12,13,16,52,48,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.97,4.51,1.51,0.0,0.0,0.0,0.0,0.0,0.0,Teddybären&Plüsch-Stadion,austria,2013-07-20,17:00:00,13,5,0,5,23,8,15,29,2,12/13
3,6413.0,Wolfsberger AC,Rapid Wien,NaN,1.0,0.0,0.0,1.33,1.33,2,2,4,2,0,2,6,3,2,0,4,0,0,2,2,2,7,13,3,6,4,7,8,26,51,49,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.29,3.37,2.33,0.0,0.0,0.0,0.0,0.0,0.0,Lavanttal Arena,austria,2013-07-20,17:00:00,9,6,0,6,20,9,11,34,0,12/13
4,7406.0,Wacker Innsbruck,Sturm Graz,NaN,1.0,0.0,0.0,0.94,1.44,2,2,4,0,0,0,7,1,2,0,4,0,0,2,0,4,15,5,5,3,10,2,21,20,48,52,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.54,3.37,2.93,0.0,0.0,0.0,0.0,0.0,0.0,Tivoli Stadion Tirol,austria,2013-07-21,14:30:00,8,6,0,6,20,8,12,41,0,12/13


In [42]:
data.isna().sum()

attendance                             30917
home_team_name                             0
away_team_name                             0
referee                                31947
Game Week                               6853
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_corner_count                     0
away_team_corner_count                     0
home_team_yellow_cards                     0
home_team_red_cards                        0
away_team_yellow_cards                     0
away_team_red_cards                        0
home_team_first_half_cards                 0
home_team_

# Attendance

In [13]:
# Pierwsze wypełnienie: średnia dla gospodarza i sezonu
data['attendance'] = data['attendance'].fillna(data.groupby(['home_team_name', 'season'])['attendance'].transform('mean'))

In [15]:
data['attendance'].isna().sum()

16647

In [16]:
# Drugie wypełnienie: średnia dla ligi i sezonu
data['attendance'] = data['attendance'].fillna(data.groupby(['league', 'season'])['attendance'].transform('mean'))

In [18]:
data['attendance'].isna().sum()

8968

In [19]:
# Trzecie wypełnienie: średnia dla ligi na przestzeni całego datasetu
data['attendance'] = data['attendance'].fillna(data.groupby(['league'])['attendance'].transform('mean'))

In [20]:
data['attendance'].isna().sum()

0

# Rolling variables

In [21]:
# Funkcja do obliczania średniej strzałów dla drużyny (zarówno dom jak i wyjazd)
def rolling_shots_avg(data, team_col, shots_col):
    return data.groupby(team_col)[shots_col].transform(lambda x: x.rolling(window=5, min_periods=1).mean())

In [22]:
# Obliczanie średniej liczby strzałów dla drużyn domowych i wyjazdowych
data['home_team_shots_rolling_avg'] = rolling_shots_avg(data, 'home_team_name', 'home_team_shots')
data['away_team_shots_rolling_avg'] = rolling_shots_avg(data, 'away_team_name', 'away_team_shots')

In [24]:
# Tworzenie nowych kolumn dla sumy kartek
data['home_team_total_cards'] = data['home_team_yellow_cards'] + data['home_team_red_cards']
data['away_team_total_cards'] = data['away_team_yellow_cards'] + data['away_team_red_cards']

In [25]:
# Funkcja do obliczania średniej ruchomej dla danych statystyk
def rolling_avg(data, team_col, stat_col):
    return data.groupby(team_col)[stat_col].transform(lambda x: x.rolling(window=5, min_periods=1).mean())

# Obliczanie średnich dla goli, rożnych i kartek
# Dla goli
data['home_team_goal_count_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_goal_count')
data['away_team_goal_count_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_goal_count')

# Dla rzutów rożnych
data['home_team_corner_count_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_corner_count')
data['away_team_corner_count_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_corner_count')

# Dla kartek
data['home_team_cards_rolling_avg'] = rolling_avg(data, 'home_team_name', 'home_team_total_cards')
data['away_team_cards_rolling_avg'] = rolling_avg(data, 'away_team_name', 'away_team_total_cards')

In [26]:
data.head()

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,home_team_shots_rolling_avg,away_team_shots_rolling_avg,home_team_total_cards,away_team_total_cards,home_team_goal_count_rolling_avg,away_team_goal_count_rolling_avg,home_team_corner_count_rolling_avg,away_team_corner_count_rolling_avg,home_team_cards_rolling_avg,away_team_cards_rolling_avg
0,8016.0,Austria Wien,Admira,NaN,1.0,0.0,0.0,1.61,0.67,2,0,2,0,0,0,6,7,2,0,5,0,0,2,1,4,13,13,7,6,6,7,9,17,53,47,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.36,5.15,9.65,0.0,0.0,0.0,0.0,0.0,0.0,Generali Arena (Wien),austria,2013-07-20,14:30:00,13,7,0,7,26,13,13,26,1,12/13,13.0,13.0,2,5,2.0,0.0,6.0,7.0,2.0,5.0
1,2840.0,Grödig,Ried,NaN,1.0,0.0,0.0,1.44,0.94,0,0,0,0,0,0,6,11,8,0,5,0,5,3,2,3,8,16,4,10,4,6,20,24,49,51,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.70,3.42,2.73,0.0,0.0,0.0,0.0,0.0,0.0,DAS.GOLDBERG Stadion,austria,2013-07-20,17:00:00,17,13,0,13,24,14,10,44,0,12/13,8.0,16.0,8,5,0.0,0.0,6.0,11.0,8.0,5.0
2,2696.0,Wiener Neustadt,Salzburg,NaN,1.0,0.0,0.0,1.11,1.94,1,5,6,3,1,2,3,10,4,0,1,0,0,4,0,1,5,18,2,6,3,12,13,16,52,48,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,6.97,4.51,1.51,0.0,0.0,0.0,0.0,0.0,0.0,Teddybären&Plüsch-Stadion,austria,2013-07-20,17:00:00,13,5,0,5,23,8,15,29,2,12/13,5.0,18.0,4,1,1.0,5.0,3.0,10.0,4.0,1.0
3,6413.0,Wolfsberger AC,Rapid Wien,NaN,1.0,0.0,0.0,1.33,1.33,2,2,4,2,0,2,6,3,2,0,4,0,0,2,2,2,7,13,3,6,4,7,8,26,51,49,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,3.29,3.37,2.33,0.0,0.0,0.0,0.0,0.0,0.0,Lavanttal Arena,austria,2013-07-20,17:00:00,9,6,0,6,20,9,11,34,0,12/13,7.0,13.0,2,4,2.0,2.0,6.0,3.0,2.0,4.0
4,7406.0,Wacker Innsbruck,Sturm Graz,NaN,1.0,0.0,0.0,0.94,1.44,2,2,4,0,0,0,7,1,2,0,4,0,0,2,0,4,15,5,5,3,10,2,21,20,48,52,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,2.54,3.37,2.93,0.0,0.0,0.0,0.0,0.0,0.0,Tivoli Stadion Tirol,austria,2013-07-21,14:30:00,8,6,0,6,20,8,12,41,0,12/13,15.0,5.0,2,4,2.0,2.0,7.0,1.0,2.0,4.0


# Game week

In [42]:
df = pd.read_csv('./data/premier league/england-premier-league-matches-2007-to-2008-stats.csv')

In [43]:
df

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1186832700,Aug 11 2007 - 11:45am,complete,NaN,Sunderland,Tottenham Hotspur,NaN,NaN,0.00,0.00,1.58,0.89,1,0,1,0,0,0,90'3,NaN,-1,-1,1,0,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stadium of Light (Sunderland),premier league
1,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,West Ham United,Manchester City,NaN,NaN,0.00,0.00,1.47,0.95,0,2,2,1,0,1,NaN,"18,87",-1,-1,0,0,4,0,0,0,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
2,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Middlesbrough,Blackburn Rovers,NaN,NaN,0.00,0.00,1.37,1.42,1,2,3,1,1,0,30,"63,79",-1,-1,3,0,4,0,2,1,0,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Riverside Stadium (Middlesbrough),premier league
3,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Everton,Wigan Athletic,NaN,NaN,0.00,0.00,1.95,0.58,2,1,3,1,1,0,"26,75",80,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Goodison Park (Liverpool),premier league
4,1186840800,Aug 11 2007 - 2:00pm,complete,NaN,Derby County,Portsmouth,NaN,NaN,0.00,0.00,0.42,1.47,2,2,4,2,1,1,"4,84","27,83",-1,-1,1,0,2,0,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pride Park Stadium (Derby),premier league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1210514400,May 11 2008 - 2:00pm,complete,NaN,West Ham United,Aston Villa,NaN,NaN,1.50,1.44,1.47,1.42,2,2,4,2,1,1,"8,88","14,58",-1,-1,2,0,2,0,2,0,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.94,61,86,56,28,9,28,78,48,86,0.0,3.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
376,1210514400,May 11 2008 - 2:00pm,complete,NaN,Tottenham Hotspur,Liverpool,NaN,NaN,1.61,1.72,1.53,1.79,0,2,2,0,0,0,NaN,"69,74",-1,-1,1,0,1,0,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,3.20,58,84,58,39,17,28,75,56,86,0.0,2.72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,White Hart Lane (London),premier league
377,1210514400,May 11 2008 - 2:00pm,complete,NaN,Sunderland,Arsenal,NaN,NaN,1.67,1.83,1.58,1.89,0,1,1,1,0,1,NaN,24,-1,-1,1,0,1,0,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,

In [46]:
import pandas as pd

# Funkcja do przetwarzania jednego pliku
def process_season(df):
    df['date_GMT'] = pd.to_datetime(df['date_GMT'], format='%b %d %Y - %I:%M%p')
    df = df.sort_values(by='date_GMT')
    if 'Game Week' not in df.columns or df['Game Week'].isna().any():
        df['Game Week'] = df['date_GMT'].rank(method='dense').astype(int)
    return df

# Przykład użycia funkcji
processed_season_df = process_season(df)
processed_season_df.to_csv('path_to_processed_season_csv_file.csv', index=False)

In [47]:
process_season = pd.read_csv('path_to_processed_season_csv_file.csv')

In [48]:
process_season

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league
0,1186832700,2007-08-11 11:45:00,complete,NaN,Sunderland,Tottenham Hotspur,NaN,1,0.00,0.00,1.58,0.89,1,0,1,0,0,0,90'3,NaN,-1,-1,1,0,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Stadium of Light (Sunderland),premier league
1,1186840800,2007-08-11 14:00:00,complete,NaN,West Ham United,Manchester City,NaN,2,0.00,0.00,1.47,0.95,0,2,2,1,0,1,NaN,"18,87",-1,-1,0,0,4,0,0,0,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Boleyn Ground (London),premier league
2,1186840800,2007-08-11 14:00:00,complete,NaN,Middlesbrough,Blackburn Rovers,NaN,2,0.00,0.00,1.37,1.42,1,2,3,1,1,0,30,"63,79",-1,-1,3,0,4,0,2,1,0,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Riverside Stadium (Middlesbrough),premier league
3,1186840800,2007-08-11 14:00:00,complete,NaN,Everton,Wigan Athletic,NaN,2,0.00,0.00,1.95,0.58,2,1,3,1,1,0,"26,75",80,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Goodison Park (Liverpool),premier league
4,1186840800,2007-08-11 14:00:00,complete,NaN,Derby County,Portsmouth,NaN,2,0.00,0.00,0.42,1.47,2,2,4,2,1,1,"4,84","27,83",-1,-1,1,0,2,0,0,1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.00,0,0,0,0,0,0,0,0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Pride Park Stadium (Derby),premier league
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,1210514400,2008-05-11 14:00:00,complete,NaN,Wigan Athletic,Manchester United,NaN,200,1.61,1.78,1.53,1.84,0,2,2,1,0,1,NaN,"33,80",-1,-1,3,0,2,0,0,3,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.28,42,64,33,17,9,28,70,31,72,0.0,3.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DW Stadium (Wigan),premier league
376,1210514400,2008-05-11 14:00:00,complete,NaN,Portsmouth,Fulham,NaN,200,1.61,0.72,1.53,0.84,0,1,1,0,0,0,NaN,76,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,2.25,45,64,31,20,11,25,58,31,70,0.0,3.90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fratton Park (Portsmouth),premier league
377,1210514400,2008-05-11 14:00:00,complete,NaN,Derby County,Reading,NaN,200,0.44,0.39,0.42,0.53,0,4,4,1,0,1,NaN,"15,61,69,90'1",-1,-1,0,0,1,0,0,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0

## Variables creation

In [46]:
data['time'] = pd.to_datetime(data['time'], format='%H:%M').dt.time
data['pora dnia'] = data['time'].apply(lambda x: 'rano' if x < pd.Timestamp("12:00").time() else ('dzień' if x < pd.Timestamp("18:00").time() else 'wieczór'))

ValueError: unconverted data remains when parsing with format "%H:%M": ":00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [49]:
data['date'] = pd.to_datetime(data['date'])
data['runda'] = data['date'].dt.month.apply(lambda x: 'wiosna' if x >= 1 and x <= 6 else 'jesień')

# Pipelines

In [49]:
df = pd.read_csv('data\premier league\england-premier-league-matches-2022-to-2023-stats.csv')

In [50]:
df = clean_data(df)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_2548\1461863474.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


In [51]:
df.sort_values('date', inplace=True)

# Goals scored before every game

In [53]:
# to jest elegancko - teraz to musze wykorzystać do pozostałych agregatów

# Tworzenie DataFrame z bramkami zdobywanymi zarówno w domu, jak i na wyjeździe
goals = pd.concat([
    df[['date', 'home_team_name', 'home_team_goal_count']].rename(columns={'home_team_name': 'team', 'home_team_goal_count': 'goals'}),
    df[['date', 'away_team_name', 'away_team_goal_count']].rename(columns={'away_team_name': 'team', 'away_team_goal_count': 'goals'})
])

# Sortowanie danych według daty, aby kumulatywne sumy były poprawne
goals.sort_values('date', inplace=True)

# Obliczanie kumulatywnej liczby bramek dla każdej drużyny
goals['cumulative_goals'] = goals.groupby('team')['goals'].cumsum()

# Usunięcie bieżących bramek z kumulatywnej sumy, aby liczyć tylko bramki przed bieżącym meczem
goals['cumulative_goals'] -= goals['goals']

# Powrót do oryginalnego DataFrame z wynikami meczów i dodanie obliczonych kumulatywnych bramek
df = df.merge(goals[['date', 'team', 'cumulative_goals']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals': 'cumulative_home_goals'}).drop('team', axis=1)
df = df.merge(goals[['date', 'team', 'cumulative_goals']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals': 'cumulative_away_goals'}).drop('team', axis=1)

# Goals conceded before every game

In [60]:
import pandas as pd

# Tworzenie DataFrame z bramkami straconymi zarówno w domu, jak i na wyjeździe
goals_conceded = pd.concat([
    df[['date', 'home_team_name', 'away_team_goal_count']].rename(columns={'home_team_name': 'team', 'away_team_goal_count': 'goals_conceded'}),
    df[['date', 'away_team_name', 'home_team_goal_count']].rename(columns={'away_team_name': 'team', 'home_team_goal_count': 'goals_conceded'})
])

# Sortowanie danych według daty
goals_conceded.sort_values('date', inplace=True)

# Obliczanie kumulatywnej liczby bramek straconych dla każdej drużyny
goals_conceded['cumulative_goals_conceded'] = goals_conceded.groupby('team')['goals_conceded'].cumsum()

# Usunięcie bieżących bramek straconych z kumulatywnej sumy, aby liczyć tylko bramki przed bieżącym meczem
goals_conceded['cumulative_goals_conceded'] -= goals_conceded['goals_conceded']

In [61]:
# Dodanie kumulatywnych bramek straconych do oryginalnego DataFrame
df = df.merge(goals_conceded[['date', 'team', 'cumulative_goals_conceded']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals_conceded': 'cumulative_home_goals_conceded'}).drop('team', axis=1)
df = df.merge(goals_conceded[['date', 'team', 'cumulative_goals_conceded']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'cumulative_goals_conceded': 'cumulative_away_goals_conceded'}).drop('team', axis=1)

In [64]:
df[(df['home_team_name'] == "Fulham") | (df['away_team_name'] == "Fulham")][['home_team_name', 'away_team_name', 'home_team_goal_count', 'away_team_goal_count', 'cumulative_home_goals', 'cumulative_away_goals', 'cumulative_home_goals_conceded', 'cumulative_away_goals_conceded']]

,home_team_name,away_team_name,home_team_goal_count,away_team_goal_count,cumulative_home_goals,cumulative_away_goals,cumulative_home_goals_conceded,cumulative_away_goals_conceded
1,Fulham,Liverpool,2,2,0,0,0,0
11,Wolverhampton Wanderers,Fulham,0,0,1,2,2,2
22,Fulham,Brentford,3,2,2,6,2,2
30,Arsenal,Fulham,2,1,9,5,2,4
41,Fulham,Brighton & Hove Albion,2,1,6,5,6,1
54,Tottenham Hotspur,Fulham,2,1,10,8,4,7
61,Nottingham Forest,Fulham,2,3,4,9,14,9
70,Fulham,Newcastle United,1,4,12,8,11,7
85,West Ham United,Fulham,3,1,5,13,9,15
89,Fulham,AFC Bournemouth,2,2,14,8,18,20


# Average total corners before game

In [65]:
import pandas as pd

# Tworzenie DataFrame z łączną liczbą rzutów rożnych zarówno w domu, jak i na wyjeździe
corners = pd.concat([
    df[['date', 'home_team_name', 'corners_total']].rename(columns={'home_team_name': 'team', 'corners_total': 'corners'}),
    df[['date', 'away_team_name', 'corners_total']].rename(columns={'away_team_name': 'team', 'corners_total': 'corners'})
])

# Sortowanie danych według daty, aby kumulatywne średnie były poprawne
corners.sort_values('date', inplace=True)

In [66]:
# Obliczanie sumy i liczby meczów
corners['cumulative_corners'] = corners.groupby('team')['corners'].cumsum()
corners['games_played'] = corners.groupby('team').cumcount() + 1  # Dodajemy 1, bo cumcount zaczyna od 0

# Obliczanie średniej kumulatywnej
corners['average_corners'] = corners['cumulative_corners'] / corners['games_played']

# Usuwamy ostatni mecz, aby uzyskać średnią przed bieżącym meczem
corners['average_corners_pre_game'] = corners.groupby('team')['average_corners'].shift().fillna(0)

In [68]:
# Dodanie średniej liczby rzutów rożnych do oryginalnego DataFrame
df = df.merge(corners[['date', 'team', 'average_corners_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_corners_pre_game': 'average_home_corners'}).drop('team', axis=1)
df = df.merge(corners[['date', 'team', 'average_corners_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_corners_pre_game': 'average_away_corners'}).drop('team', axis=1)

In [69]:
df

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals,cumulative_away_goals,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners
0,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.00,0.00,1.47,2.05,0,2,2,1,0,1,3,5,1,0,2,0,0,1,1,1,11,11,3,3,8,8,16,11,56,44,0.00,0.00,1.35,1.27,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.29,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),premier league,2022-08-05,19:00:00,8,3,0,3,22,6,16,27,2,22/23,0,0,0,0,0.000000,0.000000
1,NaN,Fulham,Liverpool,Andy Madley,1,0.00,0.00,1.53,1.21,2,2,4,1,1,0,4,4,2,0,0,0,1,1,0,0,10,12,4,4,6,8,7,9,33,67,0.00,0.00,1.23,1.50,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,9.05,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),premier league,2022-08-06,11:30:00,8,2,0,2,22,8,14,16,0,22/23,0,0,0,0,0.000000,0.000000
2,NaN,AFC Bournemouth,Aston Villa,Peter Bankes,1,0.00,0.00,1.16,1.21,2,0,2,1,1,0,5,5,3,0,3,0,2,1,2,1,6,10,4,2,2,8,18,16,34,66,0.00,0.00,0.80,1.08,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.10,3.53,2.30,1.35,1.95,3.66,7.50,1.80,1.95,Vitality Stadium (Bournemouth- Dorset),premier league,2022-08-06,14:00:00,10,6,0,6,16,6,10,34,1,22/23,0,0,0,0,0.000000,0.000000
3,NaN,Leeds United,Wolverhampton Wanderers,Robert Jones,1,0.00,0.00,1.16,0.58,2,1,3,2,1,1,6,4,2,0,0,0,1,1,0,0,14,16,4,7,10,9,13,9,40,60,0.00,0.00,1.51,1.96,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,2.30,3.58,3.07,1.34,2.03,3.62,7.50,1.80,1.95,Elland Road (Leeds- West Yorkshire),premier league,2022-08-06,14:00:00,10,2,0,2,30,11,19,22,1,22/23,0,0,0,0,0.000000,0.000000
4,52245.0,Newcastle United,Nottingham Forest,Simon Hooper,1,0.00,0.00,2.05,0.42,2,0,2,0,0,0,11,1,0,0,3,0,0,0,2,1,24,6,10,2,14,4,9,14,61,39,0.00,0.00,2.93,0.71,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.68,3.76,4.97,1.35,1.94,3.73,7.50,2.00,1.75,St. James' Park (Newcastle upon Tyne),premier league,2022-08-06,14:00:00,12,3,0,3,30,12,18,23,1,22/23,0,0,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NaN,Brentford,Manchester City,John Brooks,38,1.89,2.06,1.95,1.95,1,0,1,0,0,0,3,4,4,0,0,0,0,4,0,0,15,19,5,4,10,15,12,8,34,66,1.64,1.85,1.63,2.06,2.81,59,83,50,28,14,39,73,45,83,10.39,2.77,5.50,4.20,1.57,1.16,1.53,1.98,4.40,1.62,2.20,Gtech Community Stadium (Brentford- Middlesex),premier league,2023-

# Average corners by team before every game

In [70]:
import pandas as pd

# Tworzenie DataFrame z liczbą rzutów rożnych zarówno w domu, jak i na wyjeździe
corners = pd.concat([
    df[['date', 'home_team_name', 'home_team_corner_count']].rename(columns={'home_team_name': 'team', 'home_team_corner_count': 'corners'}),
    df[['date', 'away_team_name', 'away_team_corner_count']].rename(columns={'away_team_name': 'team', 'away_team_corner_count': 'corners'})
])

# Sortowanie danych według daty, aby kumulatywne średnie były poprawne
corners.sort_values('date', inplace=True)   

In [71]:
# Obliczanie sumy i liczby meczów
corners['cumulative_corners'] = corners.groupby('team')['corners'].cumsum()
corners['games_played'] = corners.groupby('team').cumcount() + 1  # Dodajemy 1, bo cumcount zaczyna od 0

# Obliczanie średniej kumulatywnej
corners['average_corners'] = corners['cumulative_corners'] / corners['games_played']

# Usuwamy ostatni mecz, aby uzyskać średnią przed bieżącym meczem
corners['average_corners_pre_game'] = corners.groupby('team')['average_corners'].shift().fillna(0)

In [72]:
# Dodanie średniej liczby rzutów rożnych do oryginalnego DataFrame
df = df.merge(corners[['date', 'team', 'average_corners_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_corners_pre_game': 'average_home_corners'}).drop('team', axis=1)
df = df.merge(corners[['date', 'team', 'average_corners_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_corners_pre_game': 'average_away_corners'}).drop('team', axis=1)

# Average yellow cards total before every game

In [76]:
import pandas as pd

# Tworzenie DataFrame z łączną liczbą żółtych kartek zarówno w domu, jak i na wyjeździe
cards = pd.concat([
    df[['date', 'home_team_name', 'yellow_cards_total']].rename(columns={'home_team_name': 'team', 'yellow_cards_total': 'yellow_cards'}),
    df[['date', 'away_team_name', 'yellow_cards_total']].rename(columns={'away_team_name': 'team', 'yellow_cards_total': 'yellow_cards'})
])

# Sortowanie danych według daty, aby kumulatywne średnie były poprawne
cards.sort_values('date', inplace=True)


In [77]:
# Obliczanie sumy i liczby meczów
cards['cumulative_yellow_cards'] = cards.groupby('team')['yellow_cards'].cumsum()
cards['games_played'] = cards.groupby('team').cumcount() + 1  # Dodajemy 1, bo cumcount zaczyna od 0

# Obliczanie średniej kumulatywnej
cards['average_yellow_cards'] = cards['cumulative_yellow_cards'] / cards['games_played']

# Usuwamy ostatni mecz, aby uzyskać średnią przed bieżącym meczem
cards['average_yellow_cards_pre_game'] = cards.groupby('team')['average_yellow_cards'].shift().fillna(0)


In [78]:
# Dodanie średniej liczby żółtych kartek do oryginalnego DataFrame
df = df.merge(cards[['date', 'team', 'average_yellow_cards_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_yellow_cards_pre_game': 'average_total_yellow_cards_in_home_team_games'}).drop('team', axis=1)
df = df.merge(cards[['date', 'team', 'average_yellow_cards_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_yellow_cards_pre_game': 'average_total_yellow_cards_in_away_team_games'}).drop('team', axis=1)

In [81]:
df['home_team_yellow_cards']

0      1
1      2
2      3
3      2
4      0
      ..
375    4
376    4
377    0
378    0
379    0
Name: home_team_yellow_cards, Length: 380, dtype: int64

# Average yellow cards by team before every game

In [83]:
import pandas as pd

# Tworzenie DataFrame z liczbą żółtych kartek dla gospodarzy i gości
cards = pd.concat([
    df[['date', 'home_team_name', 'home_team_yellow_cards']].rename(columns={'home_team_name': 'team', 'home_team_yellow_cards': 'yellow_cards'}),
    df[['date', 'away_team_name', 'away_team_yellow_cards']].rename(columns={'away_team_name': 'team', 'away_team_yellow_cards': 'yellow_cards'})
])

# Sortowanie danych według daty, aby kumulatywne średnie były poprawne
cards.sort_values('date', inplace=True)

In [84]:
# Obliczanie sumy i liczby meczów
cards['cumulative_yellow_cards'] = cards.groupby('team')['yellow_cards'].cumsum()
cards['games_played'] = cards.groupby('team').cumcount() + 1  # Dodajemy 1, bo cumcount zaczyna od 0

# Obliczanie średniej kumulatywnej
cards['average_yellow_cards'] = cards['cumulative_yellow_cards'] / cards['games_played']

# Usuwamy ostatni mecz, aby uzyskać średnią przed bieżącym meczem
cards['average_yellow_cards_pre_game'] = cards.groupby('team')['average_yellow_cards'].shift().fillna(0)

In [85]:
# Dodanie średniej liczby żółtych kartek do oryginalnego DataFrame
df = df.merge(cards[['date', 'team', 'average_yellow_cards_pre_game']], left_on=['date', 'home_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_yellow_cards_pre_game': 'average_yellow_cards_by_home_team_in home_team_games'}).drop('team', axis=1)
df = df.merge(cards[['date', 'team', 'average_yellow_cards_pre_game']], left_on=['date', 'away_team_name'], right_on=['date', 'team'], how='left').rename(columns={'average_yellow_cards_pre_game': 'average_yellow_cards_by_away_team_in_away_team_games'}).drop('team', axis=1)


In [86]:
df

,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,league,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,result,season,cumulative_home_goals,cumulative_away_goals,cumulative_home_goals_conceded,cumulative_away_goals_conceded,average_home_corners,average_away_corners,average_home_corners,average_away_corners,average_home_yellow_cards,average_away_yellow_cards,average_home_yellow_cards,average_away_yellow_cards
0,25286.0,Crystal Palace,Arsenal,Anthony Taylor,1,0.00,0.00,1.47,2.05,0,2,2,1,0,1,3,5,1,0,2,0,0,1,1,1,11,11,3,3,8,8,16,11,56,44,0.00,0.00,1.35,1.27,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.29,3.49,2.09,1.29,1.90,3.25,6.75,1.80,1.95,Selhurst Park (London),premier league,2022-08-05,19:00:00,8,3,0,3,22,6,16,27,2,22/23,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,Fulham,Liverpool,Andy Madley,1,0.00,0.00,1.53,1.21,2,2,4,1,1,0,4,4,2,0,0,0,1,1,0,0,10,12,4,4,6,8,7,9,33,67,0.00,0.00,1.23,1.50,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,9.05,5.40,1.34,1.15,1.66,2.43,4.10,1.91,1.91,Craven Cottage (London),premier league,2022-08-06,11:30:00,8,2,0,2,22,8,14,16,0,22/23,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,NaN,AFC Bournemouth,Aston Villa,Peter Bankes,1,0.00,0.00,1.16,1.21,2,0,2,1,1,0,5,5,3,0,3,0,2,1,2,1,6,10,4,2,2,8,18,16,34,66,0.00,0.00,0.80,1.08,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,3.10,3.53,2.30,1.35,1.95,3.66,7.50,1.80,1.95,Vitality Stadium (Bournemouth- Dorset),premier league,2022-08-06,14:00:00,10,6,0,6,16,6,10,34,1,22/23,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,NaN,Leeds United,Wolverhampton Wanderers,Robert Jones,1,0.00,0.00,1.16,0.58,2,1,3,2,1,1,6,4,2,0,0,0,1,1,0,0,14,16,4,7,10,9,13,9,40,60,0.00,0.00,1.51,1.96,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,2.30,3.58,3.07,1.34,2.03,3.62,7.50,1.80,1.95,Elland Road (Leeds- West Yorkshire),premier league,2022-08-06,14:00:00,10,2,0,2,30,11,19,22,1,22/23,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,52245.0,Newcastle United,Nottingham Forest,Simon Hooper,1,0.00,0.00,2.05,0.42,2,0,2,0,0,0,11,1,0,0,3,0,0,0,2,1,24,6,10,2,14,4,9,14,61,39,0.00,0.00,2.93,0.71,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,1.68,3.76,4.97,1.35,1.94,3.73,7.50,2.00,1.75,St. James' Park (Newcastle upon Tyne),premier league,2022-08-06,14:00:00,12,3,0,3,30,12,18,23,1,22/23,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...